<a href="https://colab.research.google.com/github/yenlung/Python-AI-Book/blob/main/%E5%BC%B7%E5%8C%96%E5%AD%B8%E7%BF%92_%E7%A8%8B%E5%BC%8F%E7%AF%87.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from tensorflow.keras.utils import to_categorical

## 抓取data

In [2]:
pip install finmind

     |████████████████████████████████| 48 kB 2.6 MB/s 
     |████████████████████████████████| 10.9 MB 34.1 MB/s 
     |████████████████████████████████| 297 kB 45.5 MB/s 
     |████████████████████████████████| 135 kB 42.3 MB/s 
     |████████████████████████████████| 95 kB 1.8 MB/s 
     |████████████████████████████████| 58 kB 3.5 MB/s 
     |████████████████████████████████| 793 kB 21.5 MB/s 
     |████████████████████████████████| 6.4 MB 38.2 MB/s 
     |████████████████████████████████| 1.1 MB 11.2 MB/s 
     |████████████████████████████████| 271 kB 11.1 MB/s 
     |████████████████████████████████| 94 kB 1.6 MB/s 
     |████████████████████████████████| 144 kB 45.5 MB/s 
     |████████████████████████████████| 96 kB 5.9 MB/s 
     |████████████████████████████████| 224 kB 47.6 MB/s 
     |████████████████████████████████| 132 kB 57.8 MB/s 
     |████████████████████████████████| 381 kB 48.9 MB/s 
     |████████████████████████████████| 130 kB 57.3 MB/s 
  Created wheel for ta:

In [3]:
# 取得股價
from FinMind.data import DataLoader

In [4]:
dl = DataLoader()
# 下載台股股價資料
stock_data = dl.taiwan_stock_daily(
    stock_id='2330', start_date='2010-01-01', end_date='2021-12-31')

In [5]:
stock_data

,date,stock_id,Trading_Volume,Trading_money,open,max,min,close,spread,Trading_turnover
0,2010-01-04,2330,39511138,2557720928,65.0,65.0,64.0,64.9,0.4,8255
1,2010-01-05,2330,38394084,2464115096,65.0,65.1,63.9,64.5,-0.4,9205
2,2010-01-06,2330,52734385,3390698544,64.5,64.9,63.7,64.9,0.4,12597
3,2010-01-07,2330,53294614,3437221996,64.9,65.0,64.2,64.2,-0.7,11195
4,2010-01-08,2330,48047497,3068341466,63.5,64.3,63.5,64.0,-0.2,9804
...,...,...,...,...,...,...,...,...,...,...
2949,2021-12-24,2330,12008673,7276331245,606.0,609.0,604.0,604.0,-2.0,10444
2950,2021-12-27,2330,16771900,10193462167,604.0,610.0,604.0,606.0,2.0,16637
2951,2021-12-28,2330,35156339,21569506434,610.0,615.0,610.0,615.0,9.0,36241
2952,2021-12-29,2330,25604320,15788043027,615.0,619.0,614.0,616.0,1.0,28804


In [6]:
#選取所需的欄位
stock_data=stock_data[['date','open','max','min','close']]

In [7]:
#重新命名表格名稱
stock_data.columns=['TIME','OPEN','MAX','MIN','CLOSE']

In [8]:
stock_data = stock_data.set_index("TIME")
stock_data

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2010-01-04,65.0,65.0,64.0,64.9
2010-01-05,65.0,65.1,63.9,64.5
2010-01-06,64.5,64.9,63.7,64.9
2010-01-07,64.9,65.0,64.2,64.2
2010-01-08,63.5,64.3,63.5,64.0
...,...,...,...,...
2021-12-24,606.0,609.0,604.0,604.0
2021-12-27,604.0,610.0,604.0,606.0
2021-12-28,610.0,615.0,610.0,615.0


In [9]:
stock_data.to_csv('stock_data.csv')

## 建立Q表設定

In [10]:
def q_table(table_name, state_data, next_state_data):
  table_name = pd.DataFrame(columns=['state','action','reward','next_state','q'])
  table_name['state'] = state_data
  table_name['next_state'] = next_state_data
  return table_name

## 交易設定

In [11]:
#actions
# Hold:0, Buy:1, Sell:2
action = [0, 1, 2]

In [12]:
def transaction(loop_num1, loop_num2, table_name):

  #交易初始設定
  hold_money=100000
  hold_money_previous=hold_money
  hold_num=0 
  hold_num_previous=0

  for i in range(loop_num1,loop_num2):
    if table_name['action'][i] ==1 and hold_money >0: # Buy
      sell_num=0
      buy_num=0
      buy_num=hold_money/table_name.state[i+1].iloc[-1]['CLOSE'] # 買入股數
      hold_money=0
      hold_num+=buy_num
      # (當期總價值-上一期價值)
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    elif table_name['action'][i] ==2: # Sell
      buy_num=0
      sell_num=hold_num
      hold_money+=sell_num * table_name.state[i+1].iloc[-1]['CLOSE'] # 把股數全數賣出

      hold_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    else: # 沒有交易
      buy_num=0
      sell_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    table_name.q[i] = table_name.reward[i]
  
  return table_name

# print('hold_money = {:.2f}'.format(hold_money))
# print(f'hold_money:{hold_money}')

In [13]:
# 修改交易
def transaction(loop_num1, loop_num2, table_name):

  #交易初始設定
  hold_money=100000
  hold_money_previous=hold_money
  hold_num=0 
  hold_num_previous=0
  buy_price=0

  for i in range(loop_num1,loop_num2):
    # if (table_name.state[i].iloc[-1]['CLOSE']-buy_price)/buy_price <= -0.1: #跌多少要賣出
    #   buy_num=0
    #   sell_num=hold_num
    #   hold_money+=sell_num * table_name.state[i+1].iloc[-1]['CLOSE'] # 把股數全數賣出
    #   hold_num=0
    #   table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
    #   hold_money_previous=hold_money
    #   hold_num_previous=hold_num
 
    if table_name['action'][i] ==1 and hold_money_previous >0: # Buy
      sell_num=0
      buy_num=0
      buy_num=hold_money/table_name.state[i+1].iloc[-1]['CLOSE'] # 買入股數
      hold_money=0
      hold_num+=buy_num
      #買入價格
      buy_price = table_name.state[i+1].iloc[-1]['CLOSE']
      # (當期總價值-上一期價值)
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    elif table_name['action'][i] ==2 and (table_name.state[i].iloc[-1]['CLOSE']-buy_price)/buy_price >= 0.02: # Sell # 漲多少才賣出
      buy_num=0
      sell_num=hold_num
      hold_money+=sell_num * table_name.state[i+1].iloc[-1]['CLOSE'] # 把股數全數賣出
      hold_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    else: # 沒有交易
      table_name['action'][i] =0
      buy_num=0
      sell_num=0
      table_name.reward[i]=(hold_money+(hold_num * table_name.state[i+1].iloc[-1]['CLOSE']))/(hold_money_previous + hold_num_previous*table_name.state[i].iloc[-1]['CLOSE'])-1
      hold_money_previous=hold_money
      hold_num_previous=hold_num

    table_name.q[i] = table_name.reward[i]
  
  return table_name

# print('hold_money = {:.2f}'.format(hold_money))
# print(f'hold_money:{hold_money}')

## Data放入Q表

In [14]:
# stock_data #2016-01-04,2021-12-30

In [15]:
state_data=[]
next_state_data=[]

In [16]:
# 資料處理成20天一筆
for i in range(len(stock_data)):
  state_data.append(stock_data.iloc[i:i+20,:])
  next_state_data.append(stock_data.iloc[i+1:i+21,:])

In [17]:
# 建立Q表：df1
df1 = q_table('q_table', state_data, next_state_data)
df1

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
1,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
3,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
4,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2950,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2951,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN
2952,OPEN MAX MIN CLOSE TIME ...,NaN,NaN,OPEN MAX MIN CLOSE TIME ...,NaN


In [18]:
df1.state[2690]

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0
2020-12-16,509.0,515.0,507.0,512.0


## Q表執行交易

In [19]:
# Q表動作：隨機動作
for i in range(len(df1)):
  df1["action"][i] = np.random.choice(3)

In [20]:
# Q表：計算reward、q
transaction(0,len(df1)-1,df1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2950,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2951,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2952,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0


In [21]:
df1

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2950,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2951,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2952,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0


## 訓練資料train

In [22]:
df1

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2949,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2950,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2951,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2952,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0


In [23]:
# df1.state[1204] #2020/12/04-2020/12/31
df1['state'][2690]
# for i in range(len(df1)):
  # df1['state'][i].iloc[-1] == '2020-12-31'

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0
2020-12-16,509.0,515.0,507.0,512.0


In [24]:
# Q表：df1
# data_train = df1[:1205].copy()

data_train = df1[:2690].copy()
# 1204:2020/12/04-2020/12/31
data_train

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [92]:
df1.state[2689]

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-03,499.5,499.5,495.0,497.0
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0


In [25]:
type(df1.state[0])

pandas.core.frame.DataFrame

In [26]:
# 正規化，當期狀態20天價格/前一期最後一天收盤價格(避免看不出價格相對)

# 2010.01.04
data_train.state[0] = data_train.state[0]/54.5 #2009年12月31日 開64.40 高64.50 低63.90 收64.50
for i in range(1,len(data_train)):
  data_train.state[i] = data_train.state[i]/df1.state[i-1].CLOSE.iloc[-1]
for i in range(0,len(data_train)):
  data_train.next_state[i] = data_train.next_state[i]/df1.state[i].CLOSE.iloc[-1]

In [27]:
# data_train：state, next_state拉平
for i in range(len(data_train.state)):
  data_train.state[i] = np.array(data_train.state[i])
  data_train.state[i] = data_train.state[i].flatten()
for i in range(len(data_train.next_state)):
  data_train.next_state[i]=np.array(data_train.next_state[i])
  data_train.next_state[i] = data_train.next_state[i].flatten()

In [28]:
data_train

,state,action,reward,next_state,q
0,"[1.1926605504587156, 1.1926605504587156, 1.174...",2,0.0,"[1.056910569105691, 1.0585365853658535, 1.0390...",0.0
1,"[1.056910569105691, 1.0585365853658535, 1.0390...",0,0.0,"[1.0785953177257526, 1.085284280936455, 1.0652...",0.0
2,"[1.0785953177257526, 1.085284280936455, 1.0652...",0,0.0,"[1.1, 1.1016949152542372, 1.088135593220339, 1...",0.0
3,"[1.1, 1.1016949152542372, 1.088135593220339, 1...",1,0.0,"[1.0618729096989967, 1.0752508361204014, 1.061...",0.0
4,"[1.0618729096989967, 1.0752508361204014, 1.061...",0,-0.032149,"[1.0829103214890017, 1.098138747884941, 1.0744...",-0.032149
...,...,...,...,...,...
2685,"[0.9577603143418467, 0.9666011787819253, 0.955...",0,0.001961,"[0.9666666666666667, 0.9676470588235294, 0.942...",0.001961
2686,"[0.9666666666666667, 0.9676470588235294, 0.942...",0,0.007828,"[0.9579256360078278, 0.958904109589041, 0.9461...",0.007828
2687,"[0.9579256360078278, 0.958904109589041, 0.9461...",0,0.0,"[0.9699029126213592, 0.970873786407767, 0.9582...",0.0
2688,"[0.9699029126213592, 0.970873786407767, 0.9582...",0,0.019417,"[0.9699029126213592, 0.9699029126213592, 0.961...",0.019417


In [29]:
# x_train=data_train[['state','action','reward','next_state']]
# y_train=data_train[['q']]

In [30]:
x_train_state = data_train.state
x_train_state 

0       [1.1926605504587156, 1.1926605504587156, 1.174...
1       [1.056910569105691, 1.0585365853658535, 1.0390...
2       [1.0785953177257526, 1.085284280936455, 1.0652...
3       [1.1, 1.1016949152542372, 1.088135593220339, 1...
4       [1.0618729096989967, 1.0752508361204014, 1.061...
                              ...                        
2685    [0.9577603143418467, 0.9666011787819253, 0.955...
2686    [0.9666666666666667, 0.9676470588235294, 0.942...
2687    [0.9579256360078278, 0.958904109589041, 0.9461...
2688    [0.9699029126213592, 0.970873786407767, 0.9582...
2689    [0.9699029126213592, 0.9699029126213592, 0.961...
Name: state, Length: 2690, dtype: object

In [31]:
# 一個一個array合成一個大array
x_train_state =np.stack(x_train_state)
x_train_state 

array([[1.19266055, 1.19266055, 1.17431193, ..., 1.12844037, 1.08990826,
        1.12844037],
       [1.05691057, 1.05853659, 1.03902439, ..., 0.98699187, 0.96747967,
        0.97235772],
       [1.07859532, 1.08528428, 1.06521739, ..., 1.01003344, 0.98662207,
        0.98662207],
       ...,
       [0.95792564, 0.95890411, 0.94618395, ..., 1.00782779, 0.99608611,
        1.00782779],
       [0.96990291, 0.97087379, 0.95825243, ..., 1.0038835 , 0.9961165 ,
        1.        ],
       [0.96990291, 0.96990291, 0.96116505, ..., 1.01941748, 0.99805825,
        1.01941748]])

In [32]:
x_train_state.shape

(2690, 80)

In [33]:
# 丟入模型，Q表動作one-hot encoding
x_train_a = to_categorical(data_train.action)

# 丟入模型，state+動作合在一起
merged_x_train = np.concatenate((x_train_state, x_train_a), axis=1) 

In [34]:
data_train.groupby('action').count()

,state,reward,next_state,q
action,,,,
0,2548,2548,2548,2548
1,58,58,58,58
2,84,84,84,84


In [35]:
x_train_a

array([[0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

In [36]:
merged_x_train

array([[1.19266055, 1.19266055, 1.17431193, ..., 0.        , 0.        ,
        1.        ],
       [1.05691057, 1.05853659, 1.03902439, ..., 1.        , 0.        ,
        0.        ],
       [1.07859532, 1.08528428, 1.06521739, ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.95792564, 0.95890411, 0.94618395, ..., 1.        , 0.        ,
        0.        ],
       [0.96990291, 0.97087379, 0.95825243, ..., 1.        , 0.        ,
        0.        ],
       [0.96990291, 0.96990291, 0.96116505, ..., 1.        , 0.        ,
        0.        ]])

In [37]:
# y
y_train = data_train.q.values
y_train

array([0.0, 0.0, 0.0, ..., 0.0, 0.01941747572815533, 0.009523809523809712],
      dtype=object)

In [38]:
np.unique(y_train)

array([-0.06919431279620847, -0.06762295081967207, -0.06339468302658491,
       ..., 0.06980802792321117, 0.07391304347826089, 0.08870967741935476],
      dtype=object)

In [39]:
# # 拉平
# y_train=y_train.ravel()

In [40]:
# float64 修改成 float32
merged_x_train = np.float32(merged_x_train)
y_train = np.float32(y_train)

## 建立Model

In [41]:
# import tensorflow as tf
# import gym
# import random
# from collections import deque 
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout, Activation, Flatten
# from tensorflow.keras.layers import LSTM, TimeDistributed, RepeatVector
from tensorflow.keras.layers import BatchNormalization

from tensorflow.keras.optimizers import Adam, RMSprop, SGD
# from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from tensorflow.keras.models import load_model 
# from imblearn.over_sampling import SMOTE

In [42]:
# x_res ,y_res = SMOTE(random_state=42).fit_resample(merged_x_train, y_train)

In [43]:
# from sklearn import linear_model

In [44]:
#get number of columns in training data
n_cols = merged_x_train.shape[1]
n_cols 

83

In [45]:
# create model
q_model = Sequential()
q_model.add(Dense(20, input_dim= n_cols, activation='relu'))
q_model.add(Dropout(0.5))

q_model.add(Dense(10, activation='relu'))
q_model.add(Dropout(0.5))

q_model.add(Dense(10, activation='relu'))
#q_model.add(Dense(15, activation='relu'))
q_model.add(BatchNormalization(momentum=0.9, epsilon=0.001))
q_model.add(Dense(1, activation='tanh'))

# q_model.add(Dense(1, activation='softmax'))

# Compile model
q_model.compile(loss='mse', optimizer=SGD(lr=0.001)) 

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/gradient_descent.py:102: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(SGD, self).__init__(name, **kwargs)


In [46]:
  # Fit the model 
  q_model.fit(merged_x_train, y_train, validation_split=0.1, epochs=50, batch_size=16, verbose=1) 

Epoch 1/50
152/152 [==============================] - 3s 6ms/step - loss: 0.4347 - val_loss: 0.0049
Epoch 2/50
152/152 [==============================] - 1s 6ms/step - loss: 0.3238 - val_loss: 0.0497
Epoch 3/50
152/152 [==============================] - 1s 4ms/step - loss: 0.1888 - val_loss: 0.0293
Epoch 4/50
152/152 [==============================] - 1s 4ms/step - loss: 0.1048 - val_loss: 0.0086
Epoch 5/50
152/152 [==============================] - 1s 4ms/step - loss: 0.0781 - val_loss: 0.0068
Epoch 6/50
152/152 [==============================] - 1s 5ms/step - loss: 0.0759 - val_loss: 0.0102
Epoch 7/50
152/152 [==============================] - 1s 6ms/step - loss: 0.0621 - val_loss: 0.0086
Epoch 8/50
152/152 [==============================] - 1s 5ms/step - loss: 0.0464 - val_loss: 0.0058
Epoch 9/50
152/152 [==============================] - 1s 5ms/step - loss: 0.0414 - val_loss: 0.0069
Epoch 10/50
152/152 [==============================] - 1s 5ms/step - loss: 0.0319 - val_loss: 0.0032

In [47]:
# #load model
# q_model=load_model("q_model_1")
# q_model = tf.keras.models.load_model('drive/MyDrive/少年Pi AI篇_寫書存放區/q_model_1')

## 計算最大q值

In [179]:
a1 = np.array([[1.,0.,0.]]*2690) #1205)
a2 = np.array([[0.,1.,0.]]*2690)
a3 = np.array([[0.,0.,1.]]*2690)

In [180]:
# data_train St+1
next_state_data =data_train.next_state
next_state_data

# 一個一個array合成一個大array
next_state_data =np.stack(next_state_data) 
next_state_data.shape

(2690, 80)

In [181]:
next_state_a1 = np.concatenate((next_state_data, a1), axis=1)
next_state_a2 = np.concatenate((next_state_data, a2), axis=1)
next_state_a3 = np.concatenate((next_state_data, a3), axis=1)

In [182]:
next_state_a1 = np.float32(next_state_a1)
next_state_a2 = np.float32(next_state_a2)
next_state_a3 = np.float32(next_state_a3)

In [183]:
next_state_a1

array([[1.0569105 , 1.0585365 , 1.0390244 , ..., 1.        , 0.        ,
        0.        ],
       [1.0785953 , 1.0852842 , 1.0652174 , ..., 1.        , 0.        ,
        0.        ],
       [1.1       , 1.101695  , 1.0881356 , ..., 1.        , 0.        ,
        0.        ],
       ...,
       [0.96990293, 0.9708738 , 0.95825243, ..., 1.        , 0.        ,
        0.        ],
       [0.96990293, 0.96990293, 0.9611651 , ..., 1.        , 0.        ,
        0.        ],
       [0.9495238 , 0.96190476, 0.947619  , ..., 1.        , 0.        ,
        0.        ]], dtype=float32)

In [184]:
next_q_a1 = q_model.predict(next_state_a1)
next_q_a2 = q_model.predict(next_state_a2)
next_q_a3 = q_model.predict(next_state_a3)

In [185]:
type(next_q_a1)

numpy.ndarray

In [186]:
# next_q_a1 = next_q_a1.flatten()
# next_q_a2 = next_q_a2.flatten()
# next_q_a3 = next_q_a3.flatten()

In [187]:
q_model_table = pd.DataFrame()
q_model_table["next_q_a1"] = next_q_a1.flatten()
q_model_table['next_q_a2'] = next_q_a2.flatten()
q_model_table['next_q_a3'] = next_q_a3.flatten()

In [188]:
q_model_table['max']=''
q_model_table['argmax']=''
for i in range(len(q_model_table)):
  e = np.array([q_model_table.next_q_a1[i],q_model_table.next_q_a2[i],q_model_table.next_q_a3[i]])
  q_model_table['max'][i] = max(e)
  q_model_table['argmax'][i] = np.argmax(e)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [189]:
q_model_table

,next_q_a1,next_q_a2,next_q_a3,max,argmax
0,-0.008525,-0.008541,-0.009192,-0.008525,0
1,-0.008637,-0.008654,-0.009304,-0.008637,0
2,-0.008760,-0.008777,-0.009427,-0.00876,0
3,-0.008672,-0.008691,-0.009340,-0.008672,0
4,-0.008687,-0.008707,-0.009355,-0.008687,0
...,...,...,...,...,...
2685,-0.008620,-0.008647,-0.009287,-0.00862,0
2686,-0.008594,-0.008623,-0.009262,-0.008594,0
2687,-0.008531,-0.008560,-0.009199,-0.008531,0
2688,-0.008533,-0.008561,-0.009200,-0.008533,0


In [190]:
q_model_table['argmax'].unique()

array([0, 1], dtype=object)

In [191]:
q_model_table.groupby('argmax').count()

,next_q_a1,next_q_a2,next_q_a3,max
argmax,,,,
0,2676,2676,2676,2676
1,14,14,14,14


## Q表更新q值

In [166]:
# Q表：df1
data_train_update = df1[:2690].copy()
# data_train = df1[:1205] #1204:2020/12/04-2020/12/31
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.032149
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [167]:
discount_rate =0.2
for i in range(len(data_train_update)):
  data_train_update.q[i]=data_train_update.reward[i]+discount_rate*q_model_table['max'][i]
  # data_train_update.action[i]=q_model_table['argmax'][i]

In [168]:
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,-0.002079
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,-0.002085
2,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,-0.002085
3,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,-0.002089
4,OPEN MAX MIN CLOSE TIME ...,0,-0.032149,OPEN MAX MIN CLOSE TIME ...,-0.034246
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,-0.000101
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.005762
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,-0.002057
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.017364


In [169]:
action_p = np.random.rand()
epsilon = 0.85
for i in range(len(data_train_update)):
  if i>0:
    if action_p < epsilon:
      # 隨機動作
      data_train_update.action[i] = np.random.choice(3)
    else:
      # 由模型選擇action
      data_train_update.action[i] = q_model_table['argmax'][i]

In [170]:
# Q表：計算reward、q
transaction(0,len(data_train_update)-1,data_train_update)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:34: RuntimeWarning: divide by zero encountered in double_scalars


,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


## 新的訓練資料(再fit Model)
 

In [171]:
data_train_update

,state,action,reward,next_state,q
0,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
1,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
3,OPEN MAX MIN CLOSE TIME ...,2,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
4,OPEN MAX MIN CLOSE TIME ...,1,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
...,...,...,...,...,...
2685,OPEN MAX MIN CLOSE TIME ...,0,0.001961,OPEN MAX MIN CLOSE TIME ...,0.001961
2686,OPEN MAX MIN CLOSE TIME ...,0,0.007828,OPEN MAX MIN CLOSE TIME ...,0.007828
2687,OPEN MAX MIN CLOSE TIME ...,0,0.0,OPEN MAX MIN CLOSE TIME ...,0.0
2688,OPEN MAX MIN CLOSE TIME ...,0,0.019417,OPEN MAX MIN CLOSE TIME ...,0.019417


In [172]:
data_train_update.state[2689]

,OPEN,MAX,MIN,CLOSE
TIME,,,,
2020-12-03,499.5,499.5,495.0,497.0
2020-12-04,498.5,505.0,497.5,503.0
2020-12-07,512.0,515.0,506.0,514.0
2020-12-08,514.0,525.0,509.0,524.0
2020-12-09,521.0,524.0,520.0,520.0
2020-12-10,511.0,515.0,510.0,512.0
2020-12-11,517.0,517.0,505.0,516.0
2020-12-14,512.0,513.0,508.0,508.0
2020-12-15,507.0,510.0,504.0,504.0


In [173]:
reward_sum=0
for i in range(len(data_train_update)):
  reward_sum+=data_train_update.reward[i]
print(reward_sum)  

2.2871343405914413


In [174]:
# 丟入模型，Q表動作one-hot encoding
x_train_a = to_categorical(data_train_update.action,3)

# # merge，x_train_state
merged_x_train = np.concatenate((x_train_state, x_train_a), axis=1) 

In [175]:
y_train = data_train_update['q'].values
# 拉平
y_train=y_train.ravel()

In [176]:
# float64 修改成 float32
merged_x_train = np.float32(merged_x_train)
y_train = np.float32(y_train)

In [177]:
# Fit the model 
q_model.fit(merged_x_train, y_train, validation_split=0.1, epochs=50, batch_size=30, verbose=1) 

Epoch 1/50
81/81 [==============================] - 0s 3ms/step - loss: 0.0014 - val_loss: 4.0819e-04
Epoch 2/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 4.7197e-04
Epoch 3/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 4.3456e-04
Epoch 4/50
81/81 [==============================] - 0s 3ms/step - loss: 0.0012 - val_loss: 4.7658e-04
Epoch 5/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 4.0728e-04
Epoch 6/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 4.1174e-04
Epoch 7/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - val_loss: 4.0899e-04
Epoch 8/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 3.7855e-04
Epoch 9/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0011 - val_loss: 4.5692e-04
Epoch 10/50
81/81 [==============================] - 0s 2ms/step - loss: 0.0012 - 

## 測試資料test

In [192]:
# df1.state[2690] 2020-12-04 - 2020-12-31
# df1.state[2709] 2020-12-31 - 2021-01-28
# df1.state[2710] 2021-01-04 - 2021-01-29

In [193]:
data_test = df1[2710:2934].copy()
data_test

,state,action,reward,next_state,q
2710,OPEN MAX MIN CLOSE TIME ...,0,0.033841,OPEN MAX MIN CLOSE TIME ...,0.033841
2711,OPEN MAX MIN CLOSE TIME ...,0,0.03437,OPEN MAX MIN CLOSE TIME ...,0.03437
2712,OPEN MAX MIN CLOSE TIME ...,0,-0.003165,OPEN MAX MIN CLOSE TIME ...,-0.003165
2713,OPEN MAX MIN CLOSE TIME ...,0,-0.004762,OPEN MAX MIN CLOSE TIME ...,-0.004762
2714,OPEN MAX MIN CLOSE TIME ...,0,0.007974,OPEN MAX MIN CLOSE TIME ...,0.007974
...,...,...,...,...,...
2929,OPEN MAX MIN CLOSE TIME ...,0,-0.0033,OPEN MAX MIN CLOSE TIME ...,-0.0033
2930,OPEN MAX MIN CLOSE TIME ...,0,0.003311,OPEN MAX MIN CLOSE TIME ...,0.003311
2931,OPEN MAX MIN CLOSE TIME ...,0,0.014851,OPEN MAX MIN CLOSE TIME ...,0.014851
2932,OPEN MAX MIN CLOSE TIME ...,0,0.001626,OPEN MAX MIN CLOSE TIME ...,0.001626


In [194]:
# data_test：state, next_state拉平
for i in range(len(data_test.state)):
  data_test.state[2710+i] = np.array(data_test.state[2710+i])
  data_test.state[2710+i] = data_test.state[2710+i].flatten()
for i in range(len(data_test.next_state)):
  data_test.next_state[2710+i]=np.array(data_test.next_state[2710+i])
  data_test.next_state[2710+i] = data_test.next_state[2710+i].flatten()

In [195]:
for i in range(len(data_test)):
  data_test.state[2710+i] = data_test.state[2710+i]/df1.state[2710+i-1].CLOSE.iloc[-1]

In [196]:
# # data_test：state, next_state拉平
# for i in range(len(data_test.state)):
#   data_test.state[1224+i] = np.array(data_test.state[1224+i])
#   data_test.state[1224+i] = data_test.state[1224+i].flatten()
# for i in range(len(data_test.next_state)):
#   data_test.next_state[1224+i]=np.array(data_test.next_state[1224+i])
#   data_test.next_state[1224+i] = data_test.next_state[1224+i].flatten()

In [197]:
# 丟入模型，調整state維度
x_test_state = data_test.state

# 一個一個array合成一個大array
x_test_state =np.stack(x_test_state)
x_test_state 

array([[0.88186356, 0.8985025 , 0.87853577, ..., 1.02995008, 0.98336106,
        0.98336106],
       [0.90693739, 0.91708968, 0.90524535, ..., 1.03553299, 0.99323181,
        1.03384095],
       [0.90834697, 0.90834697, 0.88543372, ..., 1.04418985, 1.01800327,
        1.03436989],
       ...,
       [0.9884106 , 0.99337748, 0.97847682, ..., 1.00993377, 1.        ,
        1.00331126],
       [0.98349835, 0.9950495 , 0.98019802, ..., 1.01485149, 1.00660066,
        1.01485149],
       [0.98373984, 1.        , 0.97886179, ..., 1.00650407, 0.99837398,
        1.00162602]])

In [198]:
a1 = np.array([[1.,0.,0.]]*len(x_test_state))
a2 = np.array([[0.,1.,0.]]*len(x_test_state))
a3 = np.array([[0.,0.,1.]]*len(x_test_state))

state_t_a1 = np.concatenate((x_test_state, a1), axis=1)
state_t_a2 = np.concatenate((x_test_state, a2), axis=1)
state_t_a3 = np.concatenate((x_test_state, a3), axis=1)

In [199]:
q_a1 = q_model.predict(state_t_a1) 
q_a2 = q_model.predict(state_t_a2)
q_a3 = q_model.predict(state_t_a3)
q_a1 = q_a1.flatten()
q_a2 = q_a2.flatten()
q_a3 = q_a3.flatten()

In [200]:
# test_data20天為1筆一一丟入模型，讓他選擇最大q值動作
test_table = pd.DataFrame()
test_table["q_a1"] = q_a1 # hold
test_table['q_a2'] = q_a2 # buy
test_table['q_a3'] = q_a3 # sell
test_table['max']=''
test_table['argmax']=''

for i in range(len(test_table)):
  k = np.array([test_table.q_a1[i],test_table.q_a2[i],test_table.q_a3[i]])
  test_table['max'][i] = max(k)
  test_table['argmax'][i] = np.argmax(k)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':


In [201]:
test_table

,q_a1,q_a2,q_a3,max,argmax
0,-0.008774,-0.008803,-0.009442,-0.008774,0
1,-0.008822,-0.008851,-0.009489,-0.008822,0
2,-0.008635,-0.008663,-0.009302,-0.008635,0
3,-0.008387,-0.008407,-0.009054,-0.008387,0
4,-0.008470,-0.008486,-0.009137,-0.00847,0
...,...,...,...,...,...
219,-0.008572,-0.008601,-0.009239,-0.008572,0
220,-0.008536,-0.008563,-0.009203,-0.008536,0
221,-0.008563,-0.008590,-0.009230,-0.008563,0
222,-0.008500,-0.008526,-0.009167,-0.0085,0


In [202]:
q_table_test=pd.DataFrame()
q_table_test['state'] = df1.state[2710:2934].copy()
q_table_test['action']=test_table.argmax.values
q_table_test['reward'] = ''
q_table_test['q']=''

In [203]:
# 交易
transaction(2710,2710+len(q_table_test)-1,q_table_test)

,state,action,reward,q
2710,OPEN MAX MIN CLOSE TIME ...,0,0.0,0.0
2711,OPEN MAX MIN CLOSE TIME ...,0,0.0,0.0
2712,OPEN MAX MIN CLOSE TIME ...,0,0.0,0.0
2713,OPEN MAX MIN CLOSE TIME ...,0,0.0,0.0
2714,OPEN MAX MIN CLOSE TIME ...,0,0.0,0.0
...,...,...,...,...
2929,OPEN MAX MIN CLOSE TIME ...,0,-0.0033,-0.0033
2930,OPEN MAX MIN CLOSE TIME ...,0,0.003311,0.003311
2931,OPEN MAX MIN CLOSE TIME ...,0,0.014851,0.014851
2932,OPEN MAX MIN CLOSE TIME ...,0,0.001626,0.001626


In [204]:
q_table_test.groupby('action').count()

,state,reward,q
action,,,
0,223,223,223
1,1,1,1


# save output data

In [205]:
q_table_test.to_csv('1.csv')

# save model

In [206]:
q_model.save('q_model_1')

INFO:tensorflow:Assets written to: q_model_1/assets


In [178]:
from google.colab import drive
drive.mount('/content/drive')